In [2]:
# import os
# print(os.getcwd())
# #os.chdir("./LLM")
# #print(os.getcwd())

/mnt/c/Users/cipri/OneDrive - University of Bucharest, Faculty of Mathematics and Computer Science/WORK_PROGRESS/BDD and LLM/Our


In [1]:

%load_ext autoreload
%autoreload 2
from pprint import pprint
from LLM.LLMSupport import AgentTypePrompt, LLMConvHistory
import pathlib
from typing import List, Dict, Any, Union, Tuple, Optional, Sequence
import os
#from BDDTestingLLM_args import parse_args
from BDDToLLM import *
logging.getLogger().setLevel(logging.INFO)

#args = parse_args(with_json_args=pathlib.Path(os.environ["LLM_PARAMS_PATH_INFERENCE"]))
bdd = BDDGenerationWithLLM()


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.


trainable params: 6,815,744 || all params: 8,037,076,992 || trainable%: 0.08480376642881861
Model: None


In [17]:
from BDDTestingLLM_functions import BDDAssistant, BASE_SYSTEM_MESSAGE, BASE_SYSTEM_MESSAGE_WITH_TOOLS_CALL #, hook_set_global_conversation_context #, hook_execute_tool_use_operation, hook_get_all_tool_use_operations, hook_set_global_conversation_context

bdd.llm.full_conv_history.reset()
convManager = BDDAssistant(bdd)

    
#print(BDDAssistant.instance)
#BDDAssistant.execute_tool_use_operation(tool_use_op="global_tool_get_current_test")

#convManager.do_inference("Show the current test", store_history=True, execute_tools=True)
 
# Case 1: General conversation with tools call
if True:
    convManager.do_inference("Show the current test", 
                            store_history=True, 
                                agent_mode=AgentTypePrompt.AGENT_TYPE_PROMPT_TOOL_CALL)


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


### Model inference - Prompt as text:
 <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a BDD testing expert who can write scenarios using the Gherkin language, Python and the behave library.If possible, use the tools provided first. 
If no function is suitable for the user query, answer without tools, as if you were the expert.<|eot_id|><|start_header_id|>user<|end_header_id|>

Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.

Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.Do not use variables.

{
    "type": "function",
    "function": {
        "name": "global_tool_add_new_empty_step",
        "description": "Add a new empty step to the step implementation file. The clause is the step to be added and the filepath is the path to the step

In [18]:
# Case 2: General conversation without tools call
if True:
    temp_general_conv_hist = LLMConvHistory()
    temp_general_conv_hist.set_base_system_prompt(BASE_SYSTEM_MESSAGE)
    convManager.do_inference("Tell me a joke", 
                            store_history=True, 
                            agent_mode=AgentTypePrompt.AGENT_TYPE_PROMPT_GENERAL_CONV)



from pprint import pprint
pprint(convManager.get_conv_hist().get_messages())

### Model inference - Prompt as text:
 <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a BDD testing expert who can write scenarios using the Gherkin language, Python and the behave library.If possible, use the tools provided first. 
If no function is suitable for the user query, answer without tools, as if you were the expert.<|eot_id|><|start_header_id|>user<|end_header_id|>

Show the current test<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{"name": "global_tool_get_current_test", "parameters": {}}<|eot_id|><|start_header_id|>ipython<|end_header_id|>

ToolUseOpResult(success=True, message=Scenario Outline: 
)<|eot_id|><|start_header_id|>user<|end_header_id|>

Tell me a joke<|eot_id|><|start_header_id|>assistant<|end_header_id|>


### Model inference - Outputs:
 [{'generated_text': "Here's a simple BDD test scenario for a joke:\n\n```gherkin\nFeature: Joke Teller\n  As a user\n  I want to g

In [2]:
from LLM.LLMSupport import AgentTypePrompt, LLMConvHistory


temp_prompt = """Given the functions below, is there any function description usable for the user question?
 
### Functions:
{
    name: "global_tool_get_current_test", 
    description: "Get and/or shows the current test as a string in the format of a Behave test case.\nDoes not save the test"
},

{
    name: "global_tool_get_all_tool_use_operations", 
    description: "Get all tool use operations available in the system.\nDoes not save the test"
}

### User Question: """ 


user_question = "Tell me a joke"

temp_prompt += user_question
 
#temp_prompt = temp_prompt.format() #(user_question="Tell me a joke")
agent_match_system_message = """Check the user question against the functions description above. 
Respond with a JSON format as below.
If no match is found, respond with the following JSON format:
{
    "answer": NO_MATCH,
    "matched_function": ""
}

If found, say the function name as below:
{
    "answer": MATCH,
    "matched_function": "name of the function"
}
"""

convManager.init_conv_history(override_base_system_message=agent_match_system_message)


conv_history_overridden = LLMConvHistory(base_system_prompt=agent_match_system_message)

convManager.do_inference(temp_prompt, 
                         store_history=True,                         
                         max_generated_tokens=1024,
                         agent_mode = AgentTypePrompt.AGENT_TYPE_PROMPT_TOOL_MATCH,
                         override_conv_history=conv_history_overridden,
                         )

pprint(convManager.get_conv_hist()) 

NameError: name 'convManager' is not defined

In [ ]:

hook_set_global_conversation_context(convManager)

tool_name_to_function = hook_get_all_tool_use_operations() 





    except Exception as e:
        print(f"### Error in command_test_from_user: {e}")
        convManager.pop_current_conversation()
        raise e         
    finally:
        #convManager.set_conv_hist(initial_conv)
        pass
    
    
#command_test_from_user("Save the current test to the provided path", "tool_save_current_test")
command_test_from_user("Save the current test ", "global_tool_save_current_test")
command_test_from_user("Show the current test ", "global_tool_get_current_test")
command_test_from_user("Discard the in-progress test ", "global_tool_clear_current_test")
command_test_from_user("Execute the in-progress test ", "global_tool_run_test")
#test_find_path_lists_from_user()
#test_find_path_from_user()



In [5]:
cg.prepare_data()

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

The validation split percentage 5 or dataset size 1 is too low. Please increase it. For now the script will use the entire dataset for training and validation


Running a formatting operation on the dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

Running a formatting operation on the dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

In [11]:
answer = cg.train_dataset[0]['text']

response_template = " ### Answer:"
response_idx = answer.find(response_template)
input_prompt = answer[:response_idx + len(response_template)]

input_ids = cg.tokenizer(input_prompt, return_tensors="pt").to("cuda")['input_ids'][0]
#print(input_ids)
print(len(input_ids))
#print(input_prompt)

# Next, create a chat and apply the chat template
messages = [
  {"role": "system", "content": "You are a bot that responds to weather queries."},
  {"role": "user", "content": "Hey, what's the temperature in Paris now?"}
]

messages[-1]["content"] = input_prompt 
outputs = cg.pipeline(
    messages,
    max_new_tokens=4096,
)
print(outputs[0]["generated_text"][-1])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


221
{'role': 'assistant', 'content': '{"name": "get_current_temperature", "parameters": {"location": "Paris, France"}}'}


In [13]:
cg.do_inference(input_prompt, max_generated_tokens=4096, store_history=False)

/home/ciprian/anaconda3/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/ciprian/anaconda3/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant',
 'content': '{"name": "get_current_temperature", "parameters": {"location": "Paris, France"}}'}

In [4]:
cg.do_inference("What parameters are in the following behave library given step?: @given(\"the car has (?P<engine_power>\d+) kw, weighs (?P<weight>\d+) kg, has a drag coefficient of (?P<drag>[\.\d]+)\")")

NameError: name 'cg' is not defined

In [6]:
cg.train_data

NameError: name 'cg' is not defined